# ML.Net - Recommender

In [10]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.Recommender"

In [11]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [12]:
public class MovieRating
{
    [LoadColumn(0)]
    public float userId;
    [LoadColumn(1)]
    public float movieId;
    [LoadColumn(2)]
    public float Label;
}

public class MovieRatingPrediction
{
    public float Label;
    public float Score;
}

In [13]:
// Load data
public static (IDataView training, IDataView test) LoadData(MLContext mlContext)
{
    // Load training & test datasets using datapaths
    var trainingDataPath = Path.Combine(Environment.CurrentDirectory, "Datasets\\Movies_Data", "recommendation-ratings-train.csv");
    var testDataPath = Path.Combine(Environment.CurrentDirectory, "Datasets\\Movies_Data", "recommendation-ratings-test.csv");

    IDataView trainingDataView = mlContext.Data.LoadFromTextFile<MovieRating>(trainingDataPath, hasHeader: true, separatorChar: ',');
    IDataView testDataView = mlContext.Data.LoadFromTextFile<MovieRating>(testDataPath, hasHeader: true, separatorChar: ',');

    return (trainingDataView, testDataView);
}

In [14]:
// Build and train model
public static ITransformer BuildAndTrainModel(MLContext mlContext, IDataView trainingDataView)
{
    // Add data transformations
    IEstimator<ITransformer> estimator = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "userIdEncoded", inputColumnName: "userId")
        .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "movieIdEncoded", inputColumnName: "movieId"));

    // Set algorithm options and append algorithm

    var options = new MatrixFactorizationTrainer.Options
    {
        MatrixColumnIndexColumnName = "userIdEncoded",
        MatrixRowIndexColumnName = "movieIdEncoded", 
        LabelColumnName = "Label",
        NumberOfIterations = 20,
        ApproximationRank = 100
    };
    
    var trainerEstimator = estimator.Append(mlContext.Recommendation().Trainers.MatrixFactorization(options));

    Console.WriteLine("=============== Training the model ===============");
    ITransformer model = trainerEstimator.Fit(trainingDataView);

    return model;
}

In [15]:
// Evaluate model
public static void EvaluateModel(MLContext mlContext, IDataView testDataView, ITransformer model)
{
    // Evaluate model on test data & print evaluation metrics
    Console.WriteLine("=============== Evaluating the model ===============");
    var prediction = model.Transform(testDataView);

    var metrics = mlContext.Regression.Evaluate(prediction, labelColumnName: "Label", scoreColumnName: "Score");

    Console.WriteLine("Root Mean Squared Error : " + metrics.RootMeanSquaredError.ToString());
    Console.WriteLine("RSquared: " + metrics.RSquared.ToString());
}

In [16]:
public static void UseModelForSinglePrediction(MLContext mlContext, ITransformer model)
{
    Console.WriteLine("=============== Making a prediction ===============");
    var predictionEngine = mlContext.Model.CreatePredictionEngine<MovieRating, MovieRatingPrediction>(model);

    // Create test input & make single prediction
    var testInput = new MovieRating { userId = 6, movieId = 10 };

    var movieRatingPrediction = predictionEngine.Predict(testInput);

    if (Math.Round(movieRatingPrediction.Score, 1) > 3.5)
    {
        Console.WriteLine("Movie " + testInput.movieId + " is recommended for user " + testInput.userId);
    }
    else
    {
        Console.WriteLine("Movie " + testInput.movieId + " is not recommended for user " + testInput.userId);
    }
}

In [17]:
//Save model
public static void SaveModel(MLContext mlContext, DataViewSchema trainingDataViewSchema, ITransformer model)
{
    // Save the trained model to .zip file
    var modelPath = Path.Combine(Environment.CurrentDirectory, "Datasets\\Movies_Data", "MovieRecommenderModel.zip");
    
    Console.WriteLine("=============== Saving the model to a file ===============");
    mlContext.Model.Save(model, trainingDataViewSchema, modelPath);
}

In [18]:
// MLContext to be shared across the model creation workflow objects 
MLContext mlContext = new MLContext();

// Load data
(IDataView trainingDataView, IDataView testDataView) = LoadData(mlContext);

// Build & train model
ITransformer model = BuildAndTrainModel(mlContext, trainingDataView);

// Evaluate quality of model
EvaluateModel(mlContext, testDataView, model);

// Use model to try a single prediction (one row of data)
UseModelForSinglePrediction(mlContext, model);

// Save model
SaveModel(mlContext, trainingDataView.Schema, model);


=============== Training the model ===============
=============== Evaluating the model ===============
Root Mean Squared Error : 0,9963857075088528
RSquared: 0,4097941818101922
=============== Making a prediction ===============
Movie 10 is recommended for user 6
=============== Saving the model to a file ===============
